<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/CARTAS_NOTARIALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
!pip install docxcompose
!pip install pydrive2

In [118]:
from google.colab import auth
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from datetime import datetime

from docx import Document
from docxcompose.composer import Composer
from IPython.display import display, HTML

import pandas as pd
import pickle
import re
import os

In [121]:
# Ejemplo de enlaces compartidos
shared_url_excel = 'https://docs.google.com/spreadsheets/d/10WyYPyLy3__kVQ7Vw79qZwG4gej0IxIz/edit?usp=drive_link&ouid=113057162163007358544&rtpof=true&sd=true'
shared_url_template_pc = 'https://docs.google.com/document/d/1SqavdNqkman2CKEnw-fZ0ZR2WuTI0tU-/edit?usp=drive_link&ouid=113057162163007358544&rtpof=true&sd=true'
shared_url_template_pc_fiadores = 'https://docs.google.com/document/d/1m_1CcabXddSUVzuZo2Z0d1XXJ0fc02bH/edit?usp=drive_link&ouid=113057162163007358544&rtpof=true&sd=true'
shared_url_folder_output = 'https://drive.google.com/drive/folders/1-Nxg6P6uXLAA8IURYr_Sx1RQMP7uaapH?usp=drive_link'


# Autenticación
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Función para extraer el ID del archivo de un enlace compartido de Google Drive
def extract_file_id(shared_url):
    match = re.search(r'/d/([a-zA-Z0-9_-]+)|/folders/([a-zA-Z0-9_-]+)', shared_url)
    if match:
        return match.group(1) or match.group(2)
    else:
        raise ValueError("No se pudo extraer el ID del archivo del enlace proporcionado.")


# Extraer los IDs de los archivos
file_id_excel = extract_file_id(shared_url_excel)
template_pc_id = extract_file_id(shared_url_template_pc)
template_pc_fiadores_id = extract_file_id(shared_url_template_pc_fiadores)
folder_id = extract_file_id(shared_url_folder_output)





# Inicializar contador de documentos procesados
count = 0

# Función personalizada para formatear la columna LIQUIDACIÓN
def format_liquidacion(value):
    try:
        return f"{float(value):,.2f}"
    except (ValueError, TypeError):
        return "VALOR INCORRECTO, NO ES UNA FECHA"

# Función para validar fechas
def validate_date(date):
    try:
        if pd.isnull(date):
            raise ValueError
        return pd.to_datetime(date)
    except (ValueError, TypeError):
        return datetime.now()

# Función para asegurar integridad en direcciones, distritos y provincias
def ensure_integrity(addresses, distritos, provincias):
    if not (len(addresses) == len(distritos) == len(provincias)):
        raise ValueError("Las longitudes de direcciones, distritos y provincias no coinciden.")
    return addresses, distritos, provincias

# Función para descargar y leer un archivo Excel de Google Drive
def read_excel_from_drive(file_id):
    downloadedExcel = drive.CreateFile({'id': file_id})
    downloadedExcel.FetchMetadata()
    print(f'Descargando: {downloadedExcel["title"]}')
    downloadedExcel.GetContentFile('temp.xlsx')
    df = pd.read_excel(
        'temp.xlsx',
        dtype={'CONTRATO': str},
        converters={'LIQUIDACIÓN': format_liquidacion}
    )

    # Validar la columna FECHA CARTA NOTARIAL
    df['FECHA CARTA NOTARIAL'] = df['FECHA CARTA NOTARIAL'].apply(validate_date)

    # Convertir las columnas deseadas a mayúsculas
    df['NOMBRE FIADORES'] = df['NOMBRE FIADORES'].str.upper()
    df['CLIENTE'] = df['CLIENTE'].str.upper()

    return df

# Función para descargar la plantilla de Word de Google Drive
def download_word_template(file_id, filename):
    downloadedWord = drive.CreateFile({'id': file_id})
    downloadedWord.FetchMetadata()
    print(f'Descargando plantilla: {downloadedWord["title"]}')
    downloadedWord.GetContentFile(filename)

# Función para reemplazar las palabras clave en el documento
def replace_keywords_in_doc(doc, replacements):
    for para in doc.paragraphs:
        for run in para.runs:
            for key, value in replacements.items():
                if key in run.text:
                    run.text = run.text.replace(key, str(value))
    return doc

# Función para guardar el archivo en Google Drive
def save_to_drive(filename, folder_id):
    new_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
    new_file.SetContentFile(filename)
    new_file.Upload()
    print(f'Archivo subido: {filename}')

def format_date(date):
    months = {
        1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
        5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
        9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
    }
    day = date.day
    month = months[date.month]
    year = date.year
    return f"{day} de {month} de {year}"

# Función para combinar dos documentos de Word sin perder formato
def append_docx(source, destination):
    composer = Composer(destination)
    composer.append(source)
    return destination

def clean_remove_guion(value):
    cleaned_value = value.strip()
    if cleaned_value.startswith('-'):
        cleaned_value = cleaned_value[1:].strip()  # Eliminar el guion al inicio y espacios en blanco
    return cleaned_value

# Función para procesar una dirección individual
def process_address(addr_index, address, row, combined_doc, composer, template_filename):
    cleaned_address = clean_remove_guion(address)

    replacements = {
        'P_FECHA': format_date(row['FECHA CARTA NOTARIAL']),
        'P_NOMBRE_CLIENTE': row['CLIENTE'],
        'P_DIRECCION_CLIENTE': cleaned_address,
        'P_DISTRITO_CLIENTE': clean_remove_guion(row['DISTRITO CLIENTE'].split('\n')[addr_index].strip()),
        'P_PROVINCIA_CLIENTE': clean_remove_guion(row['PROVINCIA CLIENTE'].split('\n')[addr_index].strip()),
        'P_NRO_CONTRATO': row['CONTRATO'],
        'P_TIPO_MONEDA': 'S/.' if row['MONEDA'] == 'PEN' else '$',
        'P_MONTO_LIQUIDACION': row['LIQUIDACIÓN'],
        'P_MONEDA': 'SOLES' if row['MONEDA'] == 'PEN' else 'DÓLARES',
    }

    if addr_index == 0:
        # Reemplazar palabras clave en el documento principal para la primera dirección
        replace_keywords_in_doc(combined_doc, replacements)
    else:
        # Crear una copia del documento de plantilla para direcciones adicionales
        doc_pc = Document(template_filename)
        replace_keywords_in_doc(doc_pc, replacements)

        # Combinar el documento modificado al documento principal
        composer.append(doc_pc)

# Función para procesar un fiador individual
def process_guarantor(gua_index, guarantor, row, combined_doc, composer, template_fiadores_filename):
    parts = [part.strip() for part in guarantor.split('+')]
    cleaned_guarantor = '\n'.join(parts)
    addresses_fiadores = str(row['DIRECCIÓN FIADORES']).split('\n') if pd.notna(row['DIRECCIÓN FIADORES']) else []
    distritos_fiadores = str(row['DISTRITO FIADORES']).split('\n') if pd.notna(row['DISTRITO FIADORES']) else []
    provincias_fiadores = str(row['PROVINCIA FIADORES']).split('\n') if pd.notna(row['PROVINCIA FIADORES']) else []

    if gua_index < len(addresses_fiadores):
        address_parts = addresses_fiadores[gua_index].split('/')
        distrito_parts = distritos_fiadores[gua_index].split('/')
        provincia_parts = provincias_fiadores[gua_index].split('/')

        for part_index in range(len(address_parts)):
            replacements = {
                'P_FECHA': format_date(row['FECHA CARTA NOTARIAL']),
                'P_NOMBRE_CLIENTE': row['CLIENTE'],
                'P_NOMBRE_FIADOR': clean_remove_guion(cleaned_guarantor),
                'P_DIRECCION_FIADOR': clean_remove_guion(address_parts[part_index]).strip(),
                'P_DISTRITO_FIADOR': clean_remove_guion(distrito_parts[part_index]).strip(),
                'P_PROVINCIA_FIADOR': clean_remove_guion(provincia_parts[part_index]).strip(),
                'P_NRO_CONTRATO': row['CONTRATO'],
                'P_TIPO_MONEDA': 'S/.' if row['MONEDA'] == 'PEN' else '$',
                'P_MONTO_LIQUIDACION': row['LIQUIDACIÓN'],
                'P_MONEDA': 'SOLES' if row['MONEDA'] == 'PEN' else 'DÓLARES',
            }

            # Crear una copia del documento de plantilla de fiadores
            doc_fiador = Document(template_fiadores_filename)
            replace_keywords_in_doc(doc_fiador, replacements)

            # Combinar el documento modificado al documento principal
            composer.append(doc_fiador)

            # Añadir dos saltos de línea en fiador para ocupar toda la hoja
            if len(parts) <= 1:
                if len(row['CLIENTE']) > 66: # 66 es la longitud maxima para dos lineas
                    pass  # No agregue salto
                if len(row['CLIENTE']) > 33: # 33 es la longitud maxima 1 linea
                    combined_doc.add_paragraph('\n')
                else:
                    combined_doc.add_paragraph('\n\n') # si es menor entonces agregar 2 lineas

# Función para procesar una fila individual
def process_row(index, row, folder_id):
    addresses = row['DIRECCIÓN CLIENTE'].split('\n')
    guarantors = str(row['NOMBRE FIADORES']).split('\n') if 'NOMBRE FIADORES' in row and pd.notna(row['NOMBRE FIADORES']) else []
    template_filename = 'template_pc.docx'
    template_fiadores_filename = 'template_fiadores.docx'
    # Crear un documento principal basado en la plantilla
    combined_doc = Document(template_filename)
    composer = Composer(combined_doc)

    # Procesar direcciones
    for addr_index, address in enumerate(addresses):
        process_address(addr_index, address, row, combined_doc, composer, template_filename)

    # Procesar fiadores
    for gua_index, guarantor in enumerate(guarantors):
        process_guarantor(gua_index, guarantor, row, combined_doc, composer, template_fiadores_filename)

    # Guardar el documento combinado
    filename = f'CARTA NOTARIAL RESOLUCIÓN CONTRATO {row["CONTRATO"]} - {row["CLIENTE"]}.docx'
    combined_doc.save(filename)
    save_to_drive(filename, folder_id)

# Función para procesar cada fila del DataFrame y generar documentos
def process_notarial_letter_dataframe(df_excel, folder_id):
    global count  # Para poder modificar la variable global dentro de la función
    for index, row in df_excel.iterrows():
        process_row(index, row, folder_id)
        count += 1  # Incrementar contador por cada documento procesado
    # Mostrar cantidad de documentos procesados y la URL de la carpeta con HTML
    folder_url = f'https://drive.google.com/drive/folders/{folder_id}'
    html_output = f'''
    <h1>Se han procesado {count} documentos.</h1>
    <h2>Los documentos se encuentran en la carpeta:
    <a href="{folder_url}" target="_blank">Cartas Notariales Procesadas</a></h2>
    '''
    from IPython.display import display, HTML
    display(HTML(html_output))


# Leer el archivo Excel en un DataFrame de pandas
df_excel = read_excel_from_drive(file_id_excel)


# Descargar las plantillas de Word
download_word_template(template_pc_id, 'template_pc.docx')
download_word_template(template_pc_fiadores_id, 'template_fiadores.docx')

# Continuar con el procesamiento del DataFrame
process_notarial_letter_dataframe(df_excel, folder_id)


Descargando: BaseDatosParaCartasNotariales.xlsx
Descargando plantilla: PLANTILLA_CARTA_NOTARIAL_PC.docx
Descargando plantilla: PLANTILLA_CARTA_NOTARIAL_PC_FIADOR.docx
Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 001107129600096050 - ORLANDO MEDRANO HUAMAN.docx
Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 001103359600128131 - M & R MULTISERVICIOSGENERALES Y TELECOMUNICACIONES SAC.docx
Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 001145454554849999 - PAPAYITA.docx
Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 001145454554849999 - PAPAYITA.docx
Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 001145454554849999 - PAPAYITA.docx
